In [8]:
"""
Embed tokens in the dataset
"""
# path to raw dataset
raw_dataset_path = './stlawu-webpages.jsonl'
print(raw_dataset_path)

./stlawu-webpages.jsonl


In [9]:
from datasets import load_dataset

raw_dataset = load_dataset('json', data_files=raw_dataset_path, split='train')
raw_dataset

raw_dataset['url'][:1000]


['https://www.stlawu.edu/offices/physics/observing-ao-nov-2015',
 'https://www.stlawu.edu/community-based-learning/potsdam-humane-society',
 'https://www.stlawu.edu/offices/alumni/laurentian-small-business-guide-boutique-and-home-items',
 'https://www.stlawu.edu/sites/default/files/styles/scale_2400x2400/public/2022-05/tassles.jpeg?itok=qbXRomhC',
 'https://www.stlawu.edu/article/install-minitab-express',
 'https://www.stlawu.edu/news/st-lawrence-recognized-outstanding-sophomore-support-program',
 'https://www.stlawu.edu/events?topic=115',
 'https://www.stlawu.edu/offices/academic-affairs/academic-affairs-diversity-equity-inclusion-action-plan',
 'https://www.stlawu.edu/news-digest/2025-01-28',
 'https://www.stlawu.edu/news/faculty-focus-feb-28-2022?utm_source=newsletter&utm_medium=email&utm_campaign=NewsDigest&utm_id=ND',
 'https://www.stlawu.edu/new-laurentian-guide/financial-information',
 'https://www.stlawu.edu/magazine/summer-2020',
 'https://www.stlawu.edu/people/cara-valentine-

In [10]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "intfloat/e5-mistral-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.43it/s]


In [ ]:
# import torch

# device = torch.device("cuda")
# model.to(device)

In [4]:
model.max_seq_length = 4096 * 2
print(model)
print(tokenizer)

MistralModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=2)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
    )
  )
  (norm): MistralRMSNorm((4096,), eps=1e-05)
  (rotary_emb): MistralRotaryEmbedding()
)
LlamaToken

In [5]:

tokenized_text = tokenizer(raw_dataset[0]['html_doc'])
tokenized_text.input_ids[:10]

[1, 523, 28808, 5170, 1386, 12492, 12383, 28767, 13, 28789]

In [11]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding='max_length', truncation=True, return_tensors="pt", max_length=model.max_seq_length
    )
    print(encoded_input)
    # encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    return model_output


In [ ]:
get_embeddings(raw_dataset['html_doc'][0]).last_hidden_state.shape
get_embeddings('Hi, this is me!').last_hidden_state.shape

In [ ]:
from html_cleaner import clean_html

In [ ]:
print(raw_dataset['html_doc'][0])

In [ ]:
print(clean_html(raw_dataset['html_doc'][0]))

In [ ]:
cleaned_html = clean_html(raw_dataset['html_doc'][20])

embeddings = get_embeddings(cleaned_html)
print(embeddings)


In [ ]:
print(embeddings.last_hidden_state.shape)
print(embeddings.last_hidden_state[0, :-3, :])

In [37]:
from html_cleaner import clean_dataset_path
from datasets import load_from_disk
clean_dataset = load_from_disk(clean_dataset_path)

In [38]:
print(clean_dataset)

Dataset({
    features: ['url', 'html_doc'],
    num_rows: 7657
})


In [39]:
# getting rid off unknown stuff
clean_dataset = clean_dataset.filter(lambda x: '<div>' in x['html_doc'])

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657/7657 [00:00<00:00, 100450.67 examples/s]


In [40]:
clean_dataset[4421]

{'url': 'https://www.stlawu.edu/news/black-laurentian-initiative-staff-and-alumni-present-aacu-conference?utm_source=newsletter&utm_medium=email&utm_campaign=NewsDigest&utm_id=ND',
 'html_doc': "<div>\n <div>\n  <div>\n  </div>\n  <div>\n   <div>\n    <div>\n     <div>\n     </div>\n    </div>\n   </div>\n  </div>\n  <div>\n   <main>\n    <div>\n     <article>\n      <div>\n       <div>\n       </div>\n       <div>\n        <div>\n         <h1>\n          Black Laurentian Initiative, Staff, and Alumni Present at AAC&amp;U Conference\n         </h1>\n         <div>\n          <span>\n           <time>\n            Mar 30, 2022\n           </time>\n          </span>\n         </div>\n         <div>\n          <ul>\n           <li>\n            <a>\n             Department News\n            </a>\n           </li>\n           <li>\n            <a>\n             Laurentian Connections\n            </a>\n           </li>\n           <li>\n            <a>\n             Life at St. Lawrence\n 